<a href="https://colab.research.google.com/github/insightcampus/202008-youth-bigdata/blob/master/hnjoo/sentiment_analysis/S.A_2_ML_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [감성분석 실습2] 네이버 영화리뷰 감성분석: Naive Bayes Using Sklearn
- 나이브 베이지안 정확도가 생각보다 높음

### 0. 준비하기
- 한글형태소분석기 설치하기
- 구글 드라이브 연동하기

In [1]:
# 코렙에서 Konlpy, Mecab 설치하기 
! sudo apt-get install g++ openjdk-7-jdk #Install Java 1.7+ 
#!sudo apt-get install python-dev; pip install konlpy # Python 2.x 
!sudo apt-get install python3-dev; pip3 install konlpy # Python 3.x 
!sudo apt-get install curl 
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 77 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded,

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd '/content/gdrive/My Drive/전송/movie_reviews'

/content/gdrive/My Drive/전송/movie_reviews


In [4]:
%ls

assets/  ratings_test.txt  ratings_train.txt  saved_model.pb  variables/


### 1. 데이터 가져오기 및 정제
- 데이터 다운로드 링크 : https://github.com/e9t/nsmc
- 데이터 읽기: 
  + 탭으로 구분되어 있는 텍스트 파일 > 구분기호를 의미하는 sep를 \t
  + 열의 갯수가 총 3개라 quoting=3

In [5]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv('ratings_train.txt', sep='\t', quoting=3))

In [6]:
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
# 결측치 확인하기
df.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [8]:
# 결측치 있는 행 삭제하기
df = df.dropna()

### 2. 토큰화 하기(Mecab)

In [9]:
# 토큰화 패키지 선언하기
from konlpy.tag import Mecab
mecab = Mecab()

In [10]:
# 토큰화 + 품사 부착
mecabTag = []
for doc in df['document']:
    mecabTag.append(mecab.pos(doc))
mecabTag[:3]

[[('아', 'IC'),
  ('더', 'MAG'),
  ('빙', 'MAG'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('진짜', 'MAG'),
  ('짜증', 'NNG'),
  ('나', 'NP'),
  ('네요', 'VCP+EC'),
  ('목소리', 'NNG')],
 [('흠', 'IC'),
  ('.', 'SF'),
  ('..', 'SY'),
  ('포스터', 'NNP'),
  ('보고', 'JKB'),
  ('초딩', 'NNG'),
  ('영화', 'NNG'),
  ('줄', 'NNG'),
  ('.', 'SF'),
  ('...', 'SY'),
  ('오버', 'NNG'),
  ('연기', 'NNG'),
  ('조차', 'JX'),
  ('가볍', 'VA'),
  ('지', 'EC'),
  ('않', 'VX'),
  ('구나', 'EC')],
 [('너무', 'MAG'), ('재', 'XPN'), ('밓었다그래서보는것을추천한다', 'UNKNOWN')]]

In [11]:
# 최빈어 조회 이전 list안의 list 없애기
import itertools
mecabTag_most = list(itertools.chain.from_iterable(mecabTag))
# 최빈어 조회 및 없앨 품사/단어 확인하기
from collections import Counter
Counter(mecabTag_most).most_common()

[(('.', 'SF'), 132306),
 (('영화', 'NNG'), 57614),
 (('고', 'EC'), 46412),
 (('는', 'ETM'), 45311),
 (('이', 'JKS'), 41704),
 (('의', 'JKG'), 33705),
 (('도', 'JX'), 33545),
 (('에', 'JKB'), 30526),
 (('다', 'EF'), 30201),
 (('가', 'JKS'), 28950),
 (('.', 'SY'), 27960),
 (('을', 'JKO'), 23680),
 (('..', 'SY'), 22813),
 ((',', 'SC'), 22043),
 (('는', 'JX'), 21478),
 (('들', 'XSN'), 20054),
 (('이', 'VCP'), 20049),
 (('보', 'VV'), 20032),
 (('한', 'XSA+ETM'), 18162),
 (('은', 'JX'), 17384),
 (('를', 'JKO'), 17010),
 (('게', 'EC'), 16985),
 (('!', 'SF'), 16782),
 (('하', 'XSV'), 16006),
 (('없', 'VA'), 15877),
 (('다', 'EC'), 15443),
 (('하', 'XSA'), 14002),
 (('좋', 'VA'), 13794),
 (('은', 'ETM'), 13790),
 (('?', 'SF'), 13597),
 (('하', 'VV'), 12953),
 (('지', 'EC'), 12557),
 (('었', 'EP'), 12215),
 (('너무', 'MAG'), 11000),
 (('정말', 'MAG'), 9750),
 (('는데', 'EC'), 9734),
 (('안', 'MAG'), 9599),
 (('로', 'JKB'), 9564),
 (('으로', 'JKB'), 9310),
 (('적', 'XSN'), 9294),
 (('만', 'JX'), 9242),
 (('것', 'NNB'), 9228),
 (('재밌', '

In [12]:
# 불용어 처리
stopPos = [# 'SN',          # 숫자
           # 'SL',          # 외국어
           'SE','SSO','SSC','SC','SY',  # 기호
           # 'XSN','XSV','XSA'        # 접미사
           'EP', 'EF','EC','ETN','ETM',   # 어미
           'JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX']   # 조사
stopWord = ['.']
X_train = []
for content in mecabTag:
  clear = []
  for pos in content:
    if pos[1] not in stopPos:
        if pos[0] not in stopWord:
          if len(pos[1]) > 1:
            clear.append(pos[0])
  X_train.append(' '.join(clear))
print(X_train[0:20])

['아 더 빙 진짜 짜증 나 네요 목소리', '흠 포스터 초딩 영화 줄 오버 연기 가볍 않', '너무 재 밓었다그래서보는것을추천한다', '교도소 이야기 구먼 솔직히 재미 없 평점 조정', '사이몬페그 익살 스런 연기 돋보였 영화 스파이더맨 늙 보이 했 커스틴 던스트 너무나 이뻐 보였', '막 걸음마 뗀 3 세 초등 학교 1 학년 생 인 8 살 용 영화 ㅋㅋㅋ 별반 개 아까움', '원작 긴장감 제대로 살려 내 못했', '별 반개 아깝 욕 나온다 이응경 길용우 연기 생활 몇 년 인지 정말 발 해도 그것 보단 낫 납치 감금 반복 반복 이 드라마 가족 없 연기 못 하 사람 모엿 네', '액션 없 재미 있 몇 안 되 영화', '왜 케 평점 낮 건데 ? 꽤 볼 만한데 헐리우드 식 화려 함 너무 길들여져 있 ?', '걍인피니트가짱이다 진짜 짱 이', '볼 때 눈물나 죽 90 년 대 향수 자극 ! 허진호 감성 절제 멜로 달인 이', '울 손들 횡단 보도 건널 때 뛰쳐나올 뻔 이범수 연기 드럽 못해', '담백 하 깔끔 해서 좋 신문 기사 보다 보 자꾸 잊어버린다 그 들 사람 이 것', '취향 존중 한다지만 진짜 내생 극장 본 영화 중 가장 노 잼 노 감동 임 스토리 어 거지 감동 어거지', 'ㄱ 냥 매번 긴장 되 재 밋 음 ㅠㅠ', '참 사람 들 웃긴 게 바스코 이기면 락스 코 라고 까 바비 이기 아이돌 이 깐다 그냥 까 싶 안달 난 것 보인다', '굿바이 레닌 표절 인 것 이해 하 왜 뒤 갈수록 재미없 지', '이건 정말 깨알 캐스팅 질퍽 하 않 산뜻 한 내용 구성 잘 버 러진 깨알 일 드 !', '약탈자 위한 변명 이라 저놈 들 착한 놈 들 절대 아닌걸요']


In [13]:
Y_train = list(df['label'])

### 3. sklearn input

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

### 4. NB 학습

In [15]:
count_vect = CountVectorizer()  # CountVectorizer 선언
X_train_counts = count_vect.fit_transform(X_train)  # fit and transform
clf = MultinomialNB().fit(X_train_counts, Y_train)  # MultinomialNB 선언 and fit

In [16]:
a = '와 대박 별로에요'
print(a)
# 예측
print(clf.predict(count_vect.transform([a])))
# 확률
print(clf.predict_proba(count_vect.transform([a])))

와 대박 별로에요
[1]
[[0.17241392 0.82758608]]


### 5. TEST 데이터 예측
- 데이터 정제
- 데이터 토큰화
- 불용어 처리
- 예측 모델 돌리기

In [17]:
# 데이터 불러오기
df_test = pd.DataFrame(pd.read_csv('ratings_test.txt', sep='\t', quoting=3))

In [18]:
# 데이터 결측치 행 확인하기
df_test.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [19]:
# 데이터 결측치 삭제하기
df_test = df_test.dropna()

In [20]:
df_test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [21]:
# 데이터 토큰화
mecabTag_test = []
for doc in df_test['document']:
    mecabTag_test.append(mecab.pos(doc))
mecabTag_test[:5]

[[('굳', 'NNG'), ('ㅋ', 'IC')],
 [('GDNTOPCLASSINTHECLUB', 'SL')],
 [('뭐', 'NP'),
  ('야', 'VCP+EF'),
  ('이', 'MM'),
  ('평점', 'NNG'),
  ('들', 'XSN'),
  ('은', 'JX'),
  ('.', 'SF'),
  ('...', 'SY'),
  ('나쁘', 'VA'),
  ('진', 'EC+JX'),
  ('않', 'VX'),
  ('지만', 'EC'),
  ('10', 'SN'),
  ('점', 'NNBC'),
  ('짜리', 'XSN'),
  ('는', 'JX'),
  ('더더욱', 'MAG'),
  ('아니', 'VCN'),
  ('잖아', 'EC')],
 [('지루', 'XR'),
  ('하', 'XSA'),
  ('지', 'EC'),
  ('는', 'JX'),
  ('않', 'VX'),
  ('은데', 'EC'),
  ('완전', 'NNG'),
  ('막장', 'NNG'),
  ('임', 'VCP+ETN'),
  ('.', 'SF'),
  ('..', 'SY'),
  ('돈', 'NNG'),
  ('주', 'VV'),
  ('고', 'EC'),
  ('보', 'VX'),
  ('기', 'ETN'),
  ('에', 'JKB'),
  ('는', 'JX'),
  ('.', 'SF'),
  ('...', 'SY')],
 [('3', 'SN'),
  ('D', 'SL'),
  ('만', 'JX'),
  ('아니', 'VCN'),
  ('었', 'EP'),
  ('어도', 'EC'),
  ('별', 'MM'),
  ('다섯', 'NR'),
  ('개', 'NNBC'),
  ('줬', 'VV+EP'),
  ('을', 'ETM'),
  ('텐데', 'NNB+VCP+EF'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('왜', 'MAG'),
  ('3', 'SN'),
  ('D', 'SL'),
  ('로', 'JKB'),
  ('나와서', 'VV+

In [22]:
# 불용어 처리
X_test = []
for content in mecabTag_test:
  clear = []
  for pos in content:
    if pos[1] not in stopPos:
        if pos[0] not in stopWord:
          if len(pos[1]) > 1:          # 한 글자 삭제하기
            clear.append(pos[0])
  X_test.append(' '.join(clear))
print(X_test[0:10])

['굳 ㅋ', 'GDNTOPCLASSINTHECLUB', '뭐 야 이 평점 들 나쁘 진 않 10 점 짜리 더더욱 아니', '지루 하 않 완전 막장 임 돈 주 보', '3 D 아니 별 다섯 개 줬 텐데 왜 3 D 나와서 제 심기 불편 하 하 ? ?', '음악 주 된 최고 음악 영화', '진정 한 쓰레기', '마치 미국 애니 튀어나온 듯 한 창의 력 없 로봇 디자인 고개 젖 한다', '갈수록 개판 되 가 중국 영화 유치 하 내용 없 폼 잡 끝 남 말 안 되 무기 유치 한 cg 남무 아 그립 동사서독 같 영화 이건 3 류 아류 작 이', '이별 아픔 뒤 찾아오 새로운 인연 기쁨 But 모든 사람 그렇 않']


In [23]:
Y_test = list(df_test['label'])

In [24]:
# 예측 모델 돌리기
test_y = []
for i in X_test:
  label = clf.predict(count_vect.transform([i]))
  test_y.append(label[0])

### 6. 정확도 확인

In [25]:
acc = 0
for i,label in enumerate(Y_test):
  if test_y[i] == label:
    acc += 1

acc/len(Y_test)*100

81.20487229233754

## 풀이 (by 김현진 선생님)
- 아래 코드는 강의 기반으로 작성 되었습니다.

### 1. 데이터 호출

In [26]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv('ratings_train.txt', sep='\t', quoting=3))
df = df.replace(np.nan, '', regex=True)
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [27]:
import re
# '한글'을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")
def preprocess(text):
    text = re.sub(remove_except_ko,' ',text).strip()
    return text

df['document'] = df['document'].map(lambda x : preprocess(x))

In [28]:
df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 2. 토큰화 및 품사부착

In [29]:
from konlpy.tag import Mecab
mecab = Mecab()
# mecab을 사용해서 토큰화, 한글자 제거, stop_word 제거 
stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']

# def token_mecab(text):
#     text = mecab.morphs(text)
#     text = [i for i in text if len(i)>1]
#     text = [i for i in text if i not in stop_word]
#     return text

stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',
           'SF','SE','SS','SP','SSO','SSC','SC','SY','SH','SL','SN',
           'MM','MAG','MAJ']
def token_mecab(text):
    text = mecab.pos(text)
    text = [i for i in text if len(i[0])>1]
    text = [i for i in text if i[0] not in stop_word]
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

In [30]:
token_mecab('교도소 이야기구먼 솔직히 재미는 없다')

['교도소', '이야기', '구먼', '재미']

In [31]:
# 토큰화 + 토큰리스트 생성 
def make_tokens(df):
    df['tokens'] = ''
    tokens_list=[]
    for i, row in df.iterrows():
        if i%10000==0:
            print(i,'/',len(df))
        token = token_mecab(df['document'][i])
        df['tokens'][i] = ' '.join(token)
    return df

df = make_tokens(df)

0 / 150000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


10000 / 150000
20000 / 150000
30000 / 150000
40000 / 150000
50000 / 150000
60000 / 150000
70000 / 150000
80000 / 150000
90000 / 150000
100000 / 150000
110000 / 150000
120000 / 150000
130000 / 150000
140000 / 150000


In [32]:
df.head()

,id,document,label,tokens
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,짜증 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,포스터 초딩 영화 오버 연기 가볍 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,교도소 이야기 구먼 재미 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그 익살 스런 연기 돋보였 영화 스파이더맨 보이 커스틴 던스트 이뻐 보였


### 3. 학습시키기 및 확인

In [33]:
X_train = list(df['tokens'])
Y_train = list(df['label'])

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
# CountVectorizer 선언
count_vect = CountVectorizer()
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [35]:
test = '강력한 몰입감을 선사하는 두 배우의 연기력!! 오랜만에 정말 색다른 영화였다'
test = token_mecab(test)
test = ' '.join(test)
print(test)
# 예측
print(clf.predict(count_vect.transform([test]))) 
# 확률 
print(clf.predict_proba(count_vect.transform([test])))

강력 입감 선사 배우 연기력 오랜만 색다른 영화
[1]
[[0.00266298 0.99733702]]


### 4. 예측 결과 정확도 측정

In [36]:
import pandas as pd
test_df = pd.DataFrame(pd.read_csv('ratings_test.txt', sep = '\t', quoting = 3))
test_df = test_df.replace(np.nan, '', regex = True)
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [37]:
test_df['document'] = test_df['document'].map(lambda x : preprocess(x))

In [38]:
test_df = make_tokens(test_df)
test_df.head()

0 / 50000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


10000 / 50000
20000 / 50000
30000 / 50000
40000 / 50000


,id,document,label,tokens
0,6270596,굳 ㅋ,1,
1,9274899,,0,
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편


In [39]:
predic_label = []
for i, row in test_df.iterrows():
    predic = clf.predict(count_vect.transform([test_df['tokens'][i]]))[0]
    predic_label.append(predic)
test_df['predic_label'] = predic_label
test_df.head()

,id,document,label,tokens,predic_label
0,6270596,굳 ㅋ,1,,0
1,9274899,,0,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편,0


#### confusion metrix

|표|참|거짓|
|---|---|---|
|참|TP|FP|
|거짓|FN|TN|

In [40]:
TP = FN = FP = TN = other = 0
for i in range(len(test_df)):
    if test_df['label'][i] == 1 and test_df['predic_label'][i] == 1:
        TP +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 0:
        TN +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 1:
        FN +=1
    elif test_df['label'][i] == 1 and test_df['predic_label'][i] == 0:
        FP +=1
    else:
        other +=1
print(TP,TN,FN,FP)

19633 20393 4434 5540


In [41]:
ACC = (TP+TN)/(TP+TN+FP+FN)
print('ACC: ',ACC)

ACC:  0.80052
